Time series는 temporal dimension을 가지고 있으며 financial data에서 가장 상징적 형태라 볼 수 있다. 
이번에는 신경망이 어떻게 Time series를 다루는지 볼 것이고 dl 모델이 어떻게 불확실성을 표현할 수 있는지 볼 것이다.

Time series에 들어가기 앞서 대부분 주식 시장 예측이라는 주제를 기대하겠지만 이책에서는 해당 주제를 다루지 않을 것이다.  경제학에서 시장은 효율적이다. 효율적 시장 가설은 모든 대중이 주가와 관련된 정보를 이용할 수 있다고 주장한다. 이 내용은 예측 알고리즘이 어떻게 정보를 다뤄야하는지에 대한 것으로 이어진다. 

만약 이 책에서 수익 모델을 만든다면 많은 투자자들은 이 알고리즘을 구현할 것이다. 이 알고리즘으로 인해 시장 예측에 관한 모든 변수가 바뀐다. 그렇기에 이 책에서 제시한 알고리즘이 앞으로 읽을 독자들에게는 먹히지 않을 것이다.

대신에 이 챕터에서는 위키피디아에서 트래픽 데이터를 사용할 것이다. 우리의 목표는 특정 위키피디아 페이지의 트래픽을 예측하는 것이다. wikipediatrend CRAN package에서 위키피디아 트래픽 데이터를 구할 수 있다.

여기에서 쓸 데이터셋은 145000여개의 위키피디아 페이지가 있다. 데이터를 캐글에서 구할 수 있다.

데이터셋 링크:www://www.kaggle.com/c/web-traffic-time-series-forecasting

### Visualization and preparation in pandas